### RNN y texto

Basado en el paper de Jeffrey L. Elman: **[Finding Structure in Time](https://condor.depaul.edu/dallbrit/extra/hon207/readings/elman-chapter10-finding-structure-in-time.pdf)**.

El artículo se basa en representar el tiempo en modelos de redes neuronales (conexionistas) es muy importante.que implica el uso de enlaces recurrentes para dotar a las redes de una memoria dinámica. 

En este enfoque, los patrones de unidades ocultas se retroalimentan a sí mismos, es decir las representaciones internas que se desarrollan reflejan las demandas de la tarea en el contexto de estados internos previos. Estas representaciones revelan una estructura interesante, que les permite ser altamente dependientes del contexto, al mismo tiempo que expresan generalizaciones a través de clases de elementos.

En el código siguiente:

* Función make_batch:
  - Genera los datos de entrada y salida para el entrenamiento de la RNN.
  - Convierte las oraciones en índices utilizando un diccionario (word_dict).
  - Los datos de entrada (input_batch) se crean usando codificación one-hot para representar las palabras excepto la última de cada oración.
  - El objetivo (target_batch) es la última palabra de cada oración.

* Clase TextRNN:
  - Define un modelo de RNN con una capa lineal y un sesgo para generar predicciones.
  - La función forward procesa los datos a través de la RNN y la capa lineal para obtener la predicción final.

* Entrenamiento del modelo:

  -  Se inicializan los pesos, se define la función de pérdida (CrossEntropyLoss) y el optimizador (Adam).
  -  En cada época, se procesan los lotes de datos a través del modelo, se calcula la pérdida, y se actualizan los pesos.

* Predicción y evaluación:

 - Al final del entrenamiento, se predice la última palabra de cada oración basada en las primeras dos palabras.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

def make_batch():
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()  
        input = [word_dict[n] for n in word[:-1]]  
        target = word_dict[word[-1]] 
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    # Convierte la lista de arrays de NumPy a un único array de NumPy antes de convertirlo a un tensor
    input_batch = np.array(input_batch, dtype=np.float32)
    return torch.FloatTensor(input_batch), torch.LongTensor(target_batch)

class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()
    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
    self.W = nn.Linear(n_hidden, n_class, bias=False)
    self.b = nn.Parameter(torch.ones([n_class]))

  def forward(self, hidden, X):
    X = X.transpose(0, 1) 
    salidas, hidden = self.rnn(X, hidden)
    salidas = salidas[-1] 
    model = self.W(salidas) + self.b 
    return model

### Pruebas

In [ ]:
n_step = 2 
n_hidden = 5 
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)
model = TextRNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

for epoch in range(5000):
  optimizer.zero_grad()
  hidden = torch.zeros(1, batch_size, n_hidden)
  output = model(hidden, input_batch)

  loss = criterion(output, target_batch)
  if (epoch + 1) % 1000 == 0:
    print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))

  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]
hidden = torch.zeros(1, batch_size, n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

### Ejercicios:

- Implementa métodos de regularización como dropout en la RNN para prevenir el sobreajuste.
- Además de la pérdida durante el entrenamiento, evalúa el modelo en un conjunto de datos de validación para monitorear el sobreajuste.
- Implementa métricas de rendimiento adicionales como precisión o recall.
- Experimenta con diferentes tasas de aprendizaje, tamaños de capas ocultas, y números de capas en la RNN.
- Usa técnicas como búsqueda de cuadrícula o búsqueda aleatoria para encontrar los mejores hiperparámetros.
- Normaliza las oraciones, como convertir todo a minúsculas, para hacer el modelo menos sensible a variaciones en el formato de entrada.
- Usa embeddings de palabras preentrenados como Word2Vec o GloVe en lugar de codificación one-hot para representar las palabras.
- Aumenta el conjunto de oraciones para entrenar el modelo y observa cómo mejora o empeora el rendimiento.
- Modifica la clase TextRNN para incluir dropout en la RNN y observa cómo afecta al overfitting.
- Reemplaza nn.RNN por nn.LSTM o nn.GRU para ver si mejora el rendimiento del modelo.
- Cambia el tamaño de n_hidden, el número de épocas, y la tasa de aprendizaje. Documenta cómo cada cambio afecta la pérdida y la precisión del modelo.
- Implementa un gráfico para visualizar la pérdida después de cada época usando matplotlib o seaborn.

In [ ]:
### Tu respuestas

### LSTM y texto

Aprender a almacenar información en intervalos de tiempo prolongados a través de la retropropagación recurrente lleva mucho tiempo. Las **LSTM (Long Short-Term Memory)** . LSTM es local en el espacio y  tiempo, su complejidad computacional por paso de tiempo y peso es *O(1)*. 

Este código implementa una red neuronal utilizando módulos de PyTorch para predecir la última letra de una palabra dada en base a las primeras letras de la misma. La red se basa en una arquitectura LSTM (Long Short-Term Memory), que es una variante de las redes neuronales recurrentes (RNN), adecuada para manejar dependencias a largo plazo en secuencias de datos.

Función make_batch:
 - Crea lotes de datos de entrada y salida para el entrenamiento.
 - Los datos de entrada (input_batch) se generan usando codificación one-hot para todas las letras de la palabra excepto la última.
 - El objetivo (target_batch) es el índice de la última letra de cada palabra en seq_data.

Clase TextLSTM:

 - Define el modelo LSTM con una capa lineal y un sesgo para la predicción de la letra final de cada palabra.
 - La función forward procesa los datos a través de la LSTM y la capa lineal para producir la predicción.

Entrenamiento del modelo:

 - Configura el optimizador (Adam) y la función de pérdida (CrossEntropyLoss).
 - Durante cada época, el modelo predice la salida, calcula la pérdida, realiza backpropagation, y actualiza los pesos del modelo.

Predicción:
 - Al final del entrenamiento, el modelo intenta predecir la última letra de las palabras en seq_data basándose en las tres primeras letras.

In [ ]:
def make_batch():
  input_batch, target_batch = [], []
  for seq in seq_data:
    input = [word_dict[n] for n in seq[:-1]] 
    target = word_dict[seq[-1]] 
    input_batch.append(np.eye(n_class)[input])
    target_batch.append(target)

  return input_batch, target_batch

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
    self.W = nn.Linear(n_hidden, n_class, bias=False)
    self.b = nn.Parameter(torch.ones([n_class]))

  def forward(self, X):
    input = X.transpose(0, 1)  

    hidden_state = torch.zeros(1, len(X), n_hidden)  
    cell_state = torch.zeros(1, len(X), n_hidden)   

    salidas, (_, _) = self.lstm(input, (hidden_state, cell_state))
    salidas = salidas[-1]  
    model = self.W(salidas) + self.b  
    return model


### Pruebas

In [ ]:
n_hidden = 128 

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict)  

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

model = TextLSTM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

    
for epoch in range(1000):
  optimizer.zero_grad()

  output = model(input_batch)
  loss = criterion(output, target_batch)
  if (epoch + 1) % 100 == 0:
    print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))

  loss.backward()
  optimizer.step()

inputs = [sen[:3] for sen in seq_data]

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

### Ejercicio

Repite el proceso en el caso de una red GRU.

In [ ]:
## Tu solución

### Más ejercicios

* Convierte input_batch a un tensor de PyTorch de manera más eficiente, tal como se discutió en la respuesta anterior, convirtiendo primero la lista de matrices a un único numpy.ndarray.
* Además de la pérdida, evaluar el modelo en un conjunto de validación para verificar si generaliza bien a datos no vistos.
* Usa métricas como la precisión para evaluar el rendimiento del modelo en la tarea de clasificación.
* Experimenta con diferentes configuraciones para n_hidden, la tasa de aprendizaje, y el número de capas LSTM.
* Implementa búsqueda de hiperparámetros automática para encontrar la mejor configuración.
* Añade dropout a la LSTM para reducir el overfitting si es necesario.
* Modifica la arquitectura del modelo para añadir más capas LSTM o capas lineales y observar cómo afecta el rendimiento.
* Reemplaza la codificación one-hot por embeddings de letras para ver si mejora la capacidad del modelo para captar relaciones más complejas entre caracteres.
* Aumenta seq_data con más palabras o utilizar un conjunto de datos más grande para entrenar el modelo y evaluar cómo escala con más datos.
* Implementa visualizaciones durante el entrenamiento para monitorear la pérdida y la precisión a lo largo de las épocas.



In [ ]:
## Tu respuesta

### Autoencoder recurrente

Un autoencoder recurrente es una variante especial de autoencoder diseñada para manejar secuencias de datos, como series temporales o secuencias de texto. Está compuesto por dos partes principales: un encoder y un decoder, ambos utilizando arquitecturas de red neuronal recurrente, como LSTM o GRU.

**Encoder**: Toma una secuencia de entrada y la procesa para producir un estado oculto o un conjunto de estados que representan la secuencia. Este estado actúa como una "compresión" de la información contenida en la secuencia de entrada.

**Decoder:** Utiliza el estado o estados ocultos generados por el encoder para reconstruir la secuencia de entrada. El objetivo es que la secuencia reconstruida sea lo más parecida posible a la entrada original.

El uso de autoencoders recurrentes es particularmente útil para tareas como el aprendizaje de características de secuencias sin supervisión, denoising de secuencias, y reducción de la dimensionalidad de datos secuenciales.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Definición de la clase del Autoencoder Recurrente
class RecurrentAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RecurrentAutoencoder, self).__init__()
        self.hidden_size = hidden_size

        # Encoder
        self.encoder = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

        # Decoder
        self.decoder = nn.LSTM(input_size=hidden_size, hidden_size=input_size, batch_first=True)

    def forward(self, X):
        # Encoder
        _, (hidden, _) = self.encoder(X)

        # Decoder: repetimos el estado oculto para cada paso de tiempo
        repeated_hidden = hidden.repeat(X.size(1), 1, 1).transpose(0, 1)
        decoded, _ = self.decoder(repeated_hidden)

        return decoded

# Datos de ejemplo
seq_data = ['hello world', 'deep learning', 'machine learning', 'hello there']
word_list = list(set(' '.join(seq_data).split()))
word_dict = {word: i for i, word in enumerate(word_list)}
n_class = len(word_dict)

# Crear lotes de datos
input_batch = []
for seq in seq_data:
    words = seq.split()
    input_vector = [np.eye(n_class)[word_dict[word]] for word in words]
    input_batch.append(input_vector)

# Convertir a tensores
input_batch = torch.FloatTensor(input_batch)

# Parámetros
n_hidden = 128

# Modelo
model = RecurrentAutoencoder(input_size=n_class, hidden_size=n_hidden)

# Definición de la función de pérdida y optimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Forward pass
    outputs = model(input_batch)

    # Calcular la pérdida
    loss = criterion(outputs, input_batch)

    # Backward pass y optimización
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Verificación de la reconstrucción
model.eval()
test_output = model(input_batch)
print("Original:", seq_data)
print("Reconstruido:", test_output.argmax(dim=2).numpy())  # Asumiendo uso de one-hot encoding


test_output Tensor: Al final del ciclo de entrenamiento, se pasa input_batch (el tensor que contiene las secuencias originales codificadas) a través del modelo para obtener test_output. Este tensor contiene los vectores de salida generados por el decoder del autoencoder. Cada vector de salida tiene dimensiones correspondientes al número de clases (palabras en word_dict), y cada componente del vector representa la probabilidad (o una cantidad proporcional a la probabilidad) de que una palabra específica sea la correcta para esa posición de la secuencia.

argmax(dim=2): Este método se aplica al tensor de salida para obtener los índices del valor máximo a lo largo de la dimensión específica (dim=2, que corresponde a la dimensión de las palabras en el diccionario). El resultado es un tensor donde cada elemento es el índice de la palabra con la mayor "probabilidad" (según el modelo) para cada posición en cada secuencia.

Por ejemplo, si test_output en una posición dada es `[0.1, 0.2, 0.7]` y estos valores corresponden a las probabilidades de que las palabras 'hello', 'world', 'deep' sean la palabra correcta, argmax seleccionará '2' (índice de 'deep') porque 0.7 es el valor máximo.

test_output.argmax(dim=2).numpy(): Convierte el tensor de índices a un array de NumPy para facilitar la visualización y manipulación.
Luego, el script imprime la correspondencia entre las secuencias originales y las secuencias reconstruidas según los índices predichos. En este punto, los índices se traducen nuevamente a palabras usando el diccionario inverso number_dict para mostrar las palabras reconstruidas.

Supongamos que seq_data es `['hello world', 'deep learning']` y word_dict tiene 'hello', 'world', 'deep', 'learning' mapeados a 0, 1, 2, 3 respectivamente. Si la salida es algo como:

```
Original: ['hello world', 'deep learning']
Reconstruido: [[0, 1], [2, 3]]
```

Esto indica que el modelo ha reconstruido correctamente las secuencias. Si la salida hubiese sido `[[0, 0], [2, 2]]`, esto indicaría que el modelo predijo 'hello' en lugar de 'world' y 'deep' en lugar de 'learning'.



### Ejercicios

* En lugar de utilizar codificación one-hot, que es de alta dimensionalidad y dispersa, utiliza embeddings de palabras como GloVe o Word2Vec para representar las palabras. Esto puede ayudar a capturar mejor las relaciones semánticas entre palabras y mejorar la capacidad de reconstrucción del modelo.
* Aumenta el número de capas o la cantidad de neuronas en las capas de LSTM. También puedes experimentar con diferentes arquitecturas como GRU en lugar de LSTM.
* Considera el uso de una estructura de encoder y decoder más compleja, incluyendo capas densas adicionales o técnicas de regularización como dropout.
* Experimenta con diferentes tasas de aprendizaje, algoritmos de optimización y tamaños de batch. El ajuste de estos hiperparámetros puede tener un impacto significativo en el rendimiento del modelo.
* Utiliza LSTM o GRU bidireccionales en el encoder para capturar contextos tanto futuros como pasados de la secuencia. Esto es particularmente útil para mejorar la reconstrucción en tareas donde el contexto en ambas direcciones es importante.
* Implementa técnicas de regularización como L1/L2 para reducir el sobreajuste. También puedes explorar autoencoders para denoising, donde el modelo aprende a reconstruir las secuencias originales a partir de versiones ruidosas de las mismas.
* Entrena el autoencoder con un corpus de texto más grande y evalúa su capacidad para reconstruir frases o párrafos completos. Analiza los errores de reconstrucción para entender qué tipos de estructuras o palabras son más difíciles de reconstruir.
* Modifica el dataset para introducir ruido (por ejemplo, errores tipográficos o palabras aleatorias) y entrena el autoencoder para limpiar estas secuencias. Esto puede ayudar a entender mejor cómo el modelo maneja la información esencial y cómo filtra el ruido.
* Usa el decoder del autoencoder para generar nuevas secuencias de texto. Puedes hacer esto alimentando el estado oculto del encoder con diferentes semillas o incluso estados ocultos interpolados entre múltiples ejemplos.
* Visualiza los estados ocultos generados por el encoder usando técnicas como t-SNE o PCA. Esto puede proporcionar insights sobre cómo el modelo está representando las secuencias de datos internamente.
* Implementa y compara diferentes arquitecturas de autoencoders recurrentes, como aquellos con LSTM, GRU, o capas bidireccionales, y evalúa sus diferencias en términos de rendimiento y tipo de reconstrucción que logran.

In [ ]:
## Tus respuestas

### Bi-LSTM y texto

Una red neuronal recurrente regular (RNN) se extiende a una red neuronal recurrente bidireccional (BRNN). El BRNN se puede entrenar sin la limitación de usar información de entrada solo hasta un framework futuro preestablecido. Esto se logra entrenándolo simultáneamente en la dirección del tiempo hacía adelante y atrás. 

La estructura bidireccional se puede modificar fácilmente para permitir una estimación eficiente de la probabilidad posterior condicional de secuencias de símbolos completas sin hacer ninguna suposición explícita sobre la forma de la distribución. 

El código que has proporcionado implementa una red neuronal recurrente bidireccional (BiLSTM) usando PyTorch para predecir la siguiente palabra en una secuencia basada en las palabras anteriores de una oración dada. 

Función make_batch:
 - Prepara los datos de entrada y salida para el modelo.
 - Recorre cada palabra en la oración, excepto la última, y crea un lote de entrada donde cada entrada es una secuencia de todas las palabras anteriores hasta la actual, codificadas mediante one-hot encoding y con padding de ceros para asegurar que todas las secuencias tengan la misma longitud (max_len).
 - La salida (target) es la palabra siguiente en la oración.

Clase BiLSTM:
 - Define un modelo de LSTM bidireccional. La bidireccionalidad permite al modelo aprender dependencias en ambas direcciones de la secuencia (hacia adelante y hacia atrás).
 - La capa LSTM es seguida por una capa lineal que reduce la dimensión de salida al número de clases posibles (palabras en el vocabulario).
 - La función forward procesa las entradas a través del LSTM y la capa lineal para generar predicciones.

Entrenamiento del modelo:
 - Se utiliza una función de pérdida de entropía cruzada y un optimizador Adam.
 - El modelo es entrenado por 10000 épocas, imprimiendo la pérdida cada 1000 épocas.

Predicción y salida:
 - Después del entrenamiento, el modelo intenta predecir la siguiente palabra para cada palabra de entrada en la oración, y luego se imprimen estas predicciones.

In [ ]:
def make_batch():
  input_batch = []
  target_batch = []

  words = sentence.split()
  for i, word in enumerate(words[:-1]):
    input = [word_dict[n] for n in words[:(i + 1)]]
    input = input + [0] * (max_len - len(input))
    target = word_dict[words[i + 1]]
    input_batch.append(np.eye(n_class)[input])
    target_batch.append(target)

  return input_batch, target_batch

class BiLSTM(nn.Module):
  def __init__(self):
    super(BiLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
    self.W = nn.Linear(n_hidden * 2, n_class, bias=False)
    self.b = nn.Parameter(torch.ones([n_class]))

  def forward(self, X):
    input = X.transpose(0, 1)  

    hidden_state = torch.zeros(1*2, len(X), n_hidden)   
    cell_state = torch.zeros(1*2, len(X), n_hidden)    

    salidas, (_, _) = self.lstm(input, (hidden_state, cell_state))
    salidas = salidas[-1]  
    model = self.W(salidas) + self.b 
    return model

### Pruebas

In [ ]:
n_hidden = 5 
sentence = (
        'Artificial intelligence  refers to the simulation of human intelligence '
        'is its ability to take actions that have the best chance of achieving a specific goal '
        'Deep learning techniques enable this automatic learning '
    )

word_dict = {w: i for i, w in enumerate(list(set(sentence.split())))}
number_dict = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word_dict)
max_len = len(sentence.split())

model = BiLSTM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

for epoch in range(10000):
  optimizer.zero_grad()
  output = model(input_batch)
  loss = criterion(output, target_batch)
  if (epoch + 1) % 1000 == 0:
    print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))

  loss.backward()
  optimizer.step()

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([number_dict[n.item()] for n in predict.squeeze()])

### Ejercicios:

* Implementa un conjunto de validación para evaluar el modelo durante el entrenamiento y ajustar los parámetros si es necesario para evitar el sobreajuste.
* Considera utilizar embeddings preentrenados para las palabras en lugar de codificación one-hot, lo que podría ayudar a mejorar el rendimiento capturando mejor las relaciones semánticas entre palabras.
* Experimenta con diferentes tamaños para n_hidden, tasas de aprendizaje y estructuras de red (añadiendo más capas LSTM o capas completamente conectadas).
* Incluye capas de dropout dentro de la red para ayudar a mitigar el sobreajuste, especialmente dado el número relativamente alto de épocas de entrenamiento.
* Modifica la red para incluir más o menos capas LSTM o cambia entre LSTM y GRU para ver cómo afectan al rendimiento.
* Añade Dropout y Batch Normalization a la red para observar si hay mejoras en la generalización y en la estabilidad durante el entrenamiento.
* Implementa visualizaciones de la pérdida durante el entrenamiento y las métricas de evaluación como la precisión o el recall para un conjunto de validación.
* Utiliza un corpus más grande para el entrenamiento y compara el rendimiento del modelo entrenado con más datos frente al entrenado con menos datos.

In [ ]:
## Tus respuestas